## Прогнозирование стоимости автомобиля по характеристикам


In [ ]:
# %pip install catboost
# %pip install sklearn
# %pip install tqdm
# %pip install phik
# %pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
# %pip install seaborn
# %pip install pandas-profiling[notebook]

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
# %pip freeze > requirements.txt
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))
VERSION    = 2
DIR_TRAIN  = './input/parsing-all-moscow-auto-ru-09-09-2020/' # подключил к ноутбуку внешний датасет
DIR_TEST   = './input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%    

Python       : 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
Numpy        : 1.20.3


# Data

In [2]:

# train = pd.read_csv(DIR_TRAIN+'all_auto_ru_09_09_2020.csv') # датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')
train_my = pd.read_csv('dftrain.csv') # мой датасет для обучения модели



In [13]:
# %pip uninstall pandas-profiling
# %pip install pandas-profiling[notebook,html]
# %pip install ruamel-yaml

#использование профилировщика пандас для изучения тестового датасета
# from pandas_profiling import ProfileReport
# profile = ProfileReport(train_my, title="Pandas Profiling Report for train")
# profile.to_file("your_trainreport.html")


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 10.20it/s]


## Data Preprocessing

In [17]:
# train.dropna(subset=['productionDate','mileage'], inplace=True)
# train.dropna(subset=['price'], inplace=True)
# # для baseline просто возьму пару схожих признаков без полной обработки
# columns = ['bodyType', 'brand', 'productionDate', 'engineDisplacement', 'mileage']

# df_train = train[columns]
# df_test = test[columns]
# y = train['price']

In [3]:
#из-за нехватки времени некоторые признаки в тренировочной базе являются неполными
# адаптировать тестовый датасет, чтобы там не было отсутствующих значений для  таких признаков:
mycolors=['чёрный', 'белый', 'серый', 'синий', 'серебристый','красный']
test['color']=test['color'].apply(lambda x: x if (x in mycolors) else 'другой цвет')
test['fuelType']=test['fuelType'].apply(lambda x: x if (x in ['бензин','дизель','гибрид']) else 'другое топливо')

train_my['numberOfDoors']=train_my['numberOfDoors'].apply(lambda x: int(x))


In [4]:
# 'complectation_dict','equipment_dict','model_info','super_gen',
columns_my=['bodyType', 'brand', 'color', 
        'engineDisplacement', 'enginePower', 
       'fuelType',  'mileage', 'modelDate',  'model_name',
       'name', 'numberOfDoors',  
       'productionDate', 'sell_id',  'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы',  'ПТС',
       'Привод', 'Руль']
columns_my = ['bodyType', 'brand','color',
 'engineDisplacement','enginePower', 
 'mileage',     'modelDate', 'model_name', 
  'name', 'numberOfDoors',   
  'productionDate',   'vehicleConfiguration',
]
df_train_my = train_my[columns_my]
df_test = test[columns_my]
y = train_my['price']

In [ ]:
df_train_my.sample(3)

## Label Encoding

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
# df_train['sample'] = 1 # помечаем где у нас трейн
# df_test['sample'] = 0 # помечаем где у нас тест

# data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [5]:
df_train_my['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
data_my = df_test.append(df_train_my, sort=False).reset_index(drop=True) # объединяем
# data_my.sample(3).iloc[:,:]

C:\Users\User\AppData\Local\Temp/ipykernel_13148/2061146508.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_my['sample'] = 1 # помечаем где у нас трейн
C:\Users\User\AppData\Local\Temp/ipykernel_13148/2061146508.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['sample'] = 0 # помечаем где у нас тест


In [44]:
# для катбуста не нужно собственноручно переводить в категорию
# for colum in ['bodyType', 'brand', 'engineDisplacement']:
#     data[colum] = data[colum].astype('category').cat.codes
# X = data.query('sample == 1').drop(['sample'], axis=1)
# X_sub = data.query('sample == 0').drop(['sample'], axis=1)


In [6]:
# , 'complectation_dict'
# , 'equipment_dict',
# 'mileage',
#  'model_info',
# 
# 'parsing_unixtime',
# 'sell_id',
# 'super_gen','modelDate','productionDate', 
cat_features_ids=['bodyType', 'brand', 'color',
        'engineDisplacement', 'enginePower',
       'fuelType',    'model_name','name',
        'numberOfDoors',  
         'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы',  'ПТС',
       'Привод', 'Руль']
cat_features_ids=list(set(columns_my).intersection(set(cat_features_ids)))

X = data_my.query('sample == 1').drop(['sample'], axis=1)
X_sub = data_my.query('sample == 0').drop(['sample'], axis=1)


## Train Split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели двигателя (engineDisplacement). 
C ней будем сравнивать другие модели.




In [49]:
# tmp_train = X_train.copy()
# tmp_train['price'] = y_train
# # Находим median по экземплярам engineDisplacement в трейне и размечаем тест
# predict = X_test['engineDisplacement'].map(tmp_train.groupby('engineDisplacement')['price'].median())

# #оцениваем точность
# print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

# # Model 2 : CatBoost


## Fit

### Log Traget
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    
В принциепе мы можем использовать любое приобразование на целевую переменную. Например деление на курс доллара, евро или гречки :) в дату сбора данных, смотрим дату парсинга в тесте в **parsing_unixtime**

In [ ]:
# # попытки разобраться с признаком parse_unixtime
# from datetime import datetime
# def parseunix(x):
#     ts = int(x)
#     return datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')

# df_=test.loc[:,['parsing_unixtime']] 
# df_['unixtime']=df_['parsing_unixtime'].apply(parseunix)

# df_.sort_values(by='unixtime',ascending=True)

In [8]:
print(cat_features_ids)

['color', 'enginePower', 'brand', 'bodyType', 'engineDisplacement', 'vehicleConfiguration', 'model_name', 'numberOfDoors', 'name']


In [9]:
# проверено 15.97% cat_features_ids=
# ['model_name', 'enginePower', 'brand', 'name', 'vehicleConfiguration', 'engineDisplacement', 'bodyType']
# ['bodyType', 'brand', 'name', 'enginePower', 'vehicleConfiguration', 'model_name', 'engineDisplacement', 'color']
# ['bodyType', 'brand', 'name', 'enginePower', 'vehicleConfiguration', 'numberOfDoors', 'model_name', 'engineDisplacement', 'color']
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, np.log(y_train),
         cat_features=cat_features_ids,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=20,
         use_best_model=True,
        #  plot=True
         )

model.save_model('catboost_single_model_2_baseline.model')

Learning rate set to 0.030553
0:	learn: 0.0502422	test: 0.0506036	best: 0.0506036 (0)	total: 760ms	remaining: 1h 3m 19s
20:	learn: 0.0321854	test: 0.0322138	best: 0.0322138 (20)	total: 4.97s	remaining: 19m 38s
40:	learn: 0.0233479	test: 0.0229767	best: 0.0229767 (40)	total: 8.35s	remaining: 16m 49s
60:	learn: 0.0193749	test: 0.0188143	best: 0.0188143 (60)	total: 13.7s	remaining: 18m 28s
80:	learn: 0.0174489	test: 0.0168512	best: 0.0168512 (80)	total: 17.8s	remaining: 18m
100:	learn: 0.0163539	test: 0.0157846	best: 0.0157846 (100)	total: 23.5s	remaining: 18m 58s
120:	learn: 0.0157350	test: 0.0152574	best: 0.0152574 (120)	total: 27.8s	remaining: 18m 41s
140:	learn: 0.0153433	test: 0.0149151	best: 0.0149151 (140)	total: 32.4s	remaining: 18m 37s
160:	learn: 0.0150619	test: 0.0146547	best: 0.0146547 (160)	total: 36.6s	remaining: 18m 20s
180:	learn: 0.0148210	test: 0.0144435	best: 0.0144435 (180)	total: 40.3s	remaining: 17m 53s
200:	learn: 0.0145840	test: 0.0142145	best: 0.0142145 (200)	tota

In [10]:
predict_test = np.exp(model.predict(X_test))
predict_submission = np.exp(model.predict(X_sub))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 15.86%


Как видим точность возросла до 15%, а что будет на ЛБ?

# Submission

In [17]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_myds_v{VERSION}.csv', index=False)
sample_submission.head(10)
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   sell_id  34686 non-null  int64  
 1   price    34686 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 542.1 KB


В итоге получили **MAPE 27%** на ЛБ!

Большая разница в ошибке может указывать на то что тест и трейн имеют различия по выборке или то что данные в трейне могли уже устареть и их нужно обновлять.

# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать подобрать параметры модели
* Попробовать другие алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking

Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing